# **Data Diri**


*   NIM = 2241720056
*   Nama = Dhika Wahyu Nugroho
*   Kelas = 3F
*   No Absen = 06




# **Tugas 2**



> Buatlah model klasifikasi dengan menggunakan SVM berdasarkan data pada Percobaan 5 dengan menggunakan fitur histogram.



## Mempersiapkan Data

In [ ]:
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

train_dir = "/content/drive/MyDrive/Machine_Learning/Jobsheet 4/images/test"
test_dir = "/content/drive/MyDrive/Machine_Learning/Jobsheet 4/images/training"

In [ ]:
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if not img is None:
                img_list.append((img, label))

    return img_list

In [ ]:
train_img = load_dataset(train_dir)
test_img = load_dataset(test_dir)

In [ ]:
pick_random = np.random.randint(0, len(train_img))

print(f'Image {pick_random}')
print(train_img[pick_random][0].shape)

Image 27
(555, 800, 3)


In [ ]:
def random_img_viz(img_list):
    rand_num = np.random.randint(0, len(img_list))

    img = img_list[rand_num][0]
    label = img_list[rand_num][1]
    label_str = 'day' if label == 1 else 'night'

    plt.imshow(img)
    print(f'Shape\t: {img.shape}')
    print(f'Label\t: {label}')

In [ ]:
def standarized_input(image):
    std_img = cv2.resize(image, (1100,600))

    return std_img

In [ ]:
def label_encoder(label):

    num_val = 0

    if(label == 'day'):
        num_val = 1

    return num_val

In [ ]:
def preprocess(img_list):
    std_img_list = []

    for item in img_list:
        image = item[0]
        label = item[1]

        std_img = standarized_input(image)

        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

In [ ]:
train_std_img_list = preprocess(train_img)
test_std_img_list = preprocess(test_img)

In [ ]:
pick_random = np.random.randint(0, len(train_std_img_list))

print(f'Image {pick_random}')
print(train_std_img_list[pick_random][0].shape)

Image 110
(600, 1100, 3)


## **Histogram**

In [ ]:
def extract_color_histogram(image, bins=(8, 8, 8)):
  hsv_img = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

  hist = cv2.calcHist([hsv_img], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
  cv2.normalize(hist, hist)

  return hist.flatten()

In [ ]:
def extract_histogram_feature(img_list):
    hist_features = []
    labels = []

    for img in img_list:
        hist = extract_color_histogram(img[0])
        img_label = img[1]

        hist_features.append(hist)
        labels.append(img_label)

    df = pd.DataFrame(hist_features)
    df['LABELS'] = labels

    return df

In [ ]:
train_hist_img = extract_histogram_feature(train_std_img_list)
print(f'Shape: {train_hist_img.shape}')
train_hist_img.head()

Shape: (160, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,LABELS
0,0.000403,0.006679,0.241980,0.050524,0.002353,0.001582,0.001381,0.004211,0.001116,0.098230,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0
1,0.994730,0.011372,0.000276,0.000106,0.000010,0.000008,0.000003,0.000000,0.022619,0.000759,...,0.0,0.002728,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0
2,0.107272,0.001969,0.000635,0.000193,0.000113,0.000113,0.000169,0.000956,0.159340,0.009330,...,0.0,0.003761,0.000402,0.00004,0.0,0.0,0.0,0.0,0.0,0
3,0.000000,0.000013,0.000389,0.000319,0.000223,0.000561,0.000746,0.002710,0.000096,0.009831,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0
4,0.000000,0.000000,0.000000,0.000000,0.000004,0.000036,0.000065,0.006158,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
test_hist_img = extract_histogram_feature(test_std_img_list)
print(f'Shape: {test_hist_img.shape}')
test_hist_img.head()

Shape: (240, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,LABELS
0,0.813540,0.009338,0.001473,0.001949,0.004383,0.005509,0.005379,0.038945,0.099373,0.044151,...,0.0,0.000182,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0
1,0.995323,0.004476,0.004457,0.002435,0.000648,0.000134,0.000023,0.000002,0.000637,0.039188,...,0.0,0.000028,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0
2,0.996898,0.005227,0.005873,0.002849,0.000591,0.000185,0.000090,0.000383,0.000717,0.036760,...,0.0,0.000047,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001115,0.000023,0.000162,...,0.0,0.013677,0.028539,0.001939,0.000031,0.000000,0.0,0.0,0.0,0
4,0.000000,0.000015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000820,0.000108,0.000696,...,0.0,0.012413,0.016226,0.002196,0.000162,0.000015,0.0,0.0,0.0,0




## **SVM**



In [ ]:
x_train = train_hist_img.iloc[:, :-1].values
y_train = train_hist_img.iloc[:, -1].values
x_test = test_hist_img.iloc[:, :-1].values
y_test = test_hist_img.iloc[:, -1].values

In [ ]:
model_hist = SVC(kernel='linear')
model_hist.fit(x_train, y_train)

SVC(kernel='linear')

In [ ]:
y_train_pred_hist = model_hist.predict(x_train)
y_test_pred_hist = model_hist.predict(x_test)

In [ ]:
acc_train_hist = accuracy_score(y_train, y_train_pred_hist)
acc_test_hist = accuracy_score(y_test, y_test_pred_hist)


print(f'Accuracy on train Histogram: {acc_train_hist}')
print(f'Accuracy on test Histogram: {acc_test_hist}')

Accuracy on train Histogram: 0.99375
Accuracy on test Histogram: 0.9625
